In [169]:
import pandas as pd
import os
import numpy as np
from os import listdir
from os.path import isfile, join

# testing on one file

Need to do cleaning with flag stuff. Sometimes the data isn't always good (M flag for missing). In order to deal with this, I am taking an average of previous values to make up the corrupt/non-existent data point.

I have a function, howMany and bestOne, that check how many previous values should be used to calculate the new data point. I test with a window size 1-5 to see which one yields the best result of the next 3 hours. 

In [170]:
#January 1st, 24 hours 
df = pd.read_csv("/Users/yukahatori/A_Fairness/FairnessML_git/USW00023234.csv")

#drop columns we don't need
df.drop('STATION', axis=1, inplace=True)
df.drop('NAME', axis=1, inplace=True)
df.drop('LATITUDE', axis=1, inplace=True)
df.drop('LONGITUDE', axis=1, inplace=True)
df.drop('ELEVATION', axis=1, inplace=True)
df.drop('month', axis=1, inplace=True)
df.drop('hour', axis=1, inplace=True)
df.drop('day', axis=1, inplace=True)

df

,DATE,HLY-TEMP-NORMAL,meas_flag_HLY-TEMP-NORMAL,comp_flag_HLY-TEMP-NORMAL,years_HLY-TEMP-NORMAL,HLY-TEMP-10PCTL,meas_flag_HLY-TEMP-10PCTL,comp_flag_HLY-TEMP-10PCTL,years_HLY-TEMP-10PCTL,HLY-TEMP-90PCTL,...,comp_flag_HLY-WIND-1STPCT,years_HLY-WIND-1STPCT,HLY-WIND-2NDDIR,meas_flag_HLY-WIND-2NDDIR,comp_flag_HLY-WIND-2NDDIR,years_HLY-WIND-2NDDIR,HLY-WIND-2NDPCT,meas_flag_HLY-WIND-2NDPCT,comp_flag_HLY-WIND-2NDPCT,years_HLY-WIND-2NDPCT
0,01-01T00:00:00,48.8,,S,30,44.1,,S,30,54.0,...,S,30,4.0,,S,30,14.3,,S,30
1,01-01T01:00:00,48.5,,S,30,43.0,,S,30,54.0,...,S,30,5.0,,S,30,13.1,,S,30
2,01-01T02:00:00,47.6,,S,30,42.1,,S,30,53.1,...,S,30,6.0,,S,30,12.5,,S,30
3,01-01T03:00:00,47.2,,S,30,41.0,,S,30,54.0,...,S,30,5.0,,S,30,13.5,,S,30
4,01-01T04:00:00,46.8,,S,30,39.9,,S,30,53.1,...,S,30,7.0,,S,30,11.2,,S,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12-31T19:00:00,51.5,,S,30,46.9,,S,30,55.9,...,S,30,4.0,,S,30,14.0,,S,30
8756,12-31T20:00:00,51.0,,S,30,46.0,,S,30,55.0,...,S,30,4.0,,S,30,13.8,,S,30
8757,12-31T21:00:00,50.5,,S,30,46.0,,S,30,55.0,...,S,30,4.0,,S,30,16.5,,S,30
8758,12-31T22:00:00,49.8,,S,30,45.0,,S,30,54.0,...,S,30,7.0,,S,30,14.8,,S,30


# When data is missing

NOAA has flags for missing data. X is fine, just means they rounded a non-zero to a zero value. M is bad, it means data is missing. When data is missing, fill it with a value from the average of past values. These function determine how many past values should be used to fill in the missing data. Value can range from 1 past data to 5 past datas.

In [171]:
# trying to figure out how many values of prev I should take in the average
def howMany(num, col, df):
    error = 0
    for i in range(num, 3+num):
        avg = sum(list(df[col][i-num:i]))/num
        error+= abs(df[col][i+1] - avg)
    
    return error/(300)

def bestOne(col, df):
    allResults = {}
    for i in range(1,6):
        error = howMany(i, col, df)
        allResults[error] = i
    lowest = min(allResults.keys())
    return allResults[lowest]

# bestVal = bestOne('HLY-CLOD-PCTCLR', df)
bestVal = bestOne('HLY-TEMP-NORMAL', df)
print(bestVal)

5


In [172]:
measurementFlags = []
completenessFlags = []
for column in df.columns:
    if 'meas' in column:
        measurementFlags.append(column)
    elif 'comp' in column:
        completenessFlags.append(column)

In [173]:
#checks the flags and removes any rows corresponding with 'M' which means the data is missing
#if measurement flag is == " " that means the data is fine

# same number of flags
# print(len(measurementFlags))
# print(len(completenessFlags))

#for every row, check every flag and see if their value has an invalid measurement or completeness

print(len(measurementFlags))
# X is ok = rounded down
# M is bad = missing
for flag in measurementFlags:
    print(flag)
    for i in range(len(df[flag])):
        if 'M' in df[flag][i]:
            print("i is: ", i)
            print(df[flag][i])
            avg = 0
            col = flag[10:]
            print(df[col][i-3:i+1])
            optimal_pastVals = bestOne(col, df)
            
            if i - optimal_pastVals > 0: #use the num of optimal past vals to calculate a replacement val for missing
                avg = sum(list(df[col][i-optimal_pastVals:i]))/optimal_pastVals
            elif i - 1 <= 0: #no past data to rely on 
                avg = 0
            elif i - 1 > 0:
                avg = sum(list(df[col][i-1:i]))/i
            else:
                print("something is wrong here.")
                break
            
            df.at[i, col] = avg
            print("new value: ", df[col][i])

26
meas_flag_HLY-TEMP-NORMAL
meas_flag_HLY-TEMP-10PCTL
meas_flag_HLY-TEMP-90PCTL
meas_flag_HLY-DEWP-NORMAL
meas_flag_HLY-DEWP-10PCTL
meas_flag_HLY-DEWP-90PCTL
meas_flag_HLY-PRES-NORMAL
meas_flag_HLY-PRES-10PCTL
meas_flag_HLY-PRES-90PCTL
meas_flag_HLY-CLDH-NORMAL
meas_flag_HLY-HTDH-NORMAL
meas_flag_HLY-CLOD-PCTCLR
i is:  105
M
102      21.9
103      11.6
104      10.1
105   -9999.0
Name: HLY-CLOD-PCTCLR, dtype: float64
new value:  10.1
i is:  129
M
126      21.5
127      11.3
128       9.6
129   -9999.0
Name: HLY-CLOD-PCTCLR, dtype: float64
new value:  9.6
i is:  153
M
150      21.4
151      11.4
152      10.2
153   -9999.0
Name: HLY-CLOD-PCTCLR, dtype: float64
new value:  10.2
i is:  201
M
198      22.5
199      12.9
200      12.2
201   -9999.0
Name: HLY-CLOD-PCTCLR, dtype: float64
new value:  12.2
i is:  224
M
221      25.6
222      22.9
223      13.3
224   -9999.0
Name: HLY-CLOD-PCTCLR, dtype: float64
new value:  13.3
i is:  225
M
222      22.9
223      13.3
224      13.3
225   -9999

new value:  12.2
i is:  249
M
246      13.3
247      12.2
248      12.2
249   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  12.2
i is:  273
M
270      13.0
271      11.1
272      10.2
273   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  10.2
i is:  295
M
292       9.1
293      10.3
294      13.3
295   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  13.3
i is:  297
M
294      13.3
295      13.3
296      11.7
297   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  11.7
i is:  321
M
318      12.7
319      11.4
320      11.6
321   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  11.6
i is:  5190
M
5187      13.7
5188       9.0
5189      15.9
5190   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  15.9
i is:  5214
M
5211      13.5
5212       9.7
5213      16.7
5214   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: float64
new value:  16.7
i is:  5502
M
5499      14.1
5500      11.4
5501      19.5
5502   -9999.0
Name: HLY-CLOD-PCTSCT, dtype: f

Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  31.620000000000005
i is:  295
M
292      34.9
293      31.7
294      30.4
295   -9999.0
Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  31.22
i is:  297
M
294      30.40
295      31.22
296      32.80
297   -9999.00
Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  32.20399999999999
i is:  321
M
318      29.3
319      30.0
320      32.1
321   -9999.0
Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  31.560000000000002
i is:  5190
M
5187      25.9
5188      34.0
5189      33.4
5190   -9999.0
Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  27.360000000000003
i is:  5214
M
5211      25.7
5212      33.8
5213      31.7
5214   -9999.0
Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  27.0
i is:  5502
M
5499      22.7
5500      30.2
5501      25.2
5502   -9999.0
Name: HLY-CLOD-PCTOVC, dtype: float64
new value:  23.060000000000002
i is:  5503
M
5500      30.20
5501      25.20
5502      23.06
5503   -9999.00
Name: HLY-CLOD-PCTOVC, dtype: floa

In [188]:
def checkMeasurement(df):
#     print(len(measurementFlags))
    # X is ok = rounded down
    # M is bad = missing
    for flag in measurementFlags:
#         print(flag)
        for i in range(len(df[flag])):
            if 'M' in df[flag][i]:
#                 print("i is: ", i)
#                 print(df[flag][i])
                avg = 0
                col = flag[10:]
#                 print(df[col][i-3:i+1])
                optimal_pastVals = bestOne(col, df)

                if i - optimal_pastVals > 0: #use the num of optimal past vals to calculate a replacement val for missing
                    avg = sum(list(df[col][i-optimal_pastVals:i]))/optimal_pastVals
                elif i - 1 <= 0: #no past data to rely on 
                    avg = 0
                elif i - 1 > 0:
                    avg = sum(list(df[col][i-1:i]))/i
                else:
                    print("something is wrong here.")
                    return

                df.at[i, col] = avg
#                 print("new value: ", df[col][i])
    return df

In [175]:
print(len(completenessFlags))
# X is ok = rounded down
# M is bad = missing
for flag in completenessFlags:
    print(flag)
    for i in range(len(df[flag])):
        if 'S' not in df[flag][i]:
            print("----------")
            print("i is: ", i)
            
            
            avg = 0
            col = flag[10:]
            print(df[col][i-3:i]) #printing out the vals of the HLY because you can see what data vals are
            col = 'years_' + col
            optimal_pastVals = bestOne(col, df)
            print("optimal val: ", optimal_pastVals)
            if i - optimal_pastVals > 0: #use the num of optimal past vals to calculate a replacement val for missing
                sumOf = sum(list(df[col][i-optimal_pastVals:i]))
                avg = float((sumOf)/(optimal_pastVals))
                print("avg is: ", avg)
            elif i - 1 <= 0: #no past data to rely on 
                avg = 0
            elif i - 1 > 0:
                avg = sum(list(df[col][i-1:i]))/i
            else:
                print("something is wrong here.")
                break
            
            df.at[i, col] = avg
            print("new value: ", df[col][i])
            print("----------")

26
comp_flag_HLY-TEMP-NORMAL
comp_flag_HLY-TEMP-10PCTL
comp_flag_HLY-TEMP-90PCTL
comp_flag_HLY-DEWP-NORMAL
comp_flag_HLY-DEWP-10PCTL
comp_flag_HLY-DEWP-90PCTL
comp_flag_HLY-PRES-NORMAL
comp_flag_HLY-PRES-10PCTL
comp_flag_HLY-PRES-90PCTL
comp_flag_HLY-CLDH-NORMAL
comp_flag_HLY-HTDH-NORMAL
comp_flag_HLY-CLOD-PCTCLR
----------
i is:  105
102    21.9
103    11.6
104    10.1
Name: HLY-CLOD-PCTCLR, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  129
126    21.5
127    11.3
128     9.6
Name: HLY-CLOD-PCTCLR, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  153
150    21.4
151    11.4
152    10.2
Name: HLY-CLOD-PCTCLR, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  201
198    22.5
199    12.9
200    12.2
Name: HLY-CLOD-PCTCLR, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  224
221    25.6
222    22.9
223    13.3
Name: HLY-CLOD-PCTC

optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  5214
5211    15.3
5212    19.3
5213    18.9
Name: HLY-CLOD-PCTBKN, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  5502
5499    17.6
5500    20.9
5501    20.6
Name: HLY-CLOD-PCTBKN, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  5503
5500    20.90
5501    20.60
5502    20.75
Name: HLY-CLOD-PCTBKN, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  5574
5571    18.2
5572    21.8
5573    20.2
Name: HLY-CLOD-PCTBKN, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  5575
5572    21.8
5573    20.2
5574    21.0
Name: HLY-CLOD-PCTBKN, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
----------
i is:  5576
5573    20.2
5574    21.0
5575    20.6
Name: HLY-CLOD-PCTBKN, dtype: float64
optimal val:  5
avg is:  30.0
new value:  30
----------
-----

comp_flag_HLY-WIND-1STDIR
comp_flag_HLY-WIND-1STPCT
comp_flag_HLY-WIND-2NDDIR
comp_flag_HLY-WIND-2NDPCT


In [189]:
def checkCompleteness(df):
#     print(len(completenessFlags))
    # X is ok = rounded down
    # M is bad = missing
    for flag in completenessFlags:
#         print(flag)
        for i in range(len(df[flag])):
            if 'S' not in df[flag][i]:
#                 print("----------")
#                 print("i is: ", i)
                avg = 0
                col = flag[10:]
                print(df[col][i-3:i]) #printing out the vals of the HLY because you can see what data vals are
                col = 'years_' + col
                optimal_pastVals = bestOne(col, df)
#                 print("optimal val: ", optimal_pastVals)
                if i - optimal_pastVals > 0: #use the num of optimal past vals to calculate a replacement val for missing
                    sumOf = sum(list(df[col][i-optimal_pastVals:i]))
                    avg = float((sumOf)/(optimal_pastVals))
#                     print("avg is: ", avg)
                elif i - 1 <= 0: #no past data to rely on 
                    avg = 0
                elif i - 1 > 0:
                    avg = sum(list(df[col][i-1:i]))/i
                else:
                    print("something is wrong here.")
                    return

                df.at[i, col] = avg
#                 print("new value: ", df[col][i])
#                 print("----------")
    return df

In [190]:
# #January 1st, 24 hours 
# df = pd.read_csv("/Users/yukahatori/A_Fairness/FairnessML_git/USW00023234.csv")

#checks the flags and removes any rows corresponding with 'M' which means the data is missing
        
#get rid of all measurement and completeness flags
df = df[df.columns.drop(list(df.filter(regex='meas')))]
df = df[df.columns.drop(list(df.filter(regex='comp')))]


In [191]:
#rename columns
df.columns = ['date', 'HLY_TEMP_NORMAL', 'years_HLY_TEMP_NORMAL', 'HLY_TEMP_10PCTL',
       'years_HLY_TEMP_10PCTL', 'HLY_TEMP_90PCTL', 'years_HLY_TEMP_90PCTL',
       'HLY_DEWP_NORMAL', 'years_HLY_DEWP_NORMAL', 'HLY_DEWP_10PCTL',
       'years_HLY_DEWPv10PCTL', 'HLY_DEWP_90PCTL', 'years_HLY_DEWP_90PCTL',
       'HLY_PRES_NORMAL', 'years_HLY_PRES_NORMAL', 'HLY_PRES_10PCTL',
       'years_HLY_PRES_10PCTL', 'HLY_PRES_90PCTL', 'years_HLY_PRES_90PCTL',
       'HLY_CLDH_NORMAL', 'years_HLY_CLDH_NORMAL', 'HLY_HTDH_NORMAL',
       'years_HLY_HTDH_NORMAL', 'HLY_CLOD_PCTCLR', 'years_HLY_CLOD_PCTCLR',
       'HLY_CLOD_PCTFEW', 'years_HLY_CLOD_PCTFEW', 'HLY_CLOD_PCTSCT',
       'years_HLY_CLOD_PCTSCT', 'HLY_CLOD_PCTBKN', 'years_HLY_CLOD_PCTBKN',
       'HLY_CLOD_PCTOVC', 'years_HLY_CLOD_PCTOVC', 'HLY_HIDX_NORMAL',
       'years_HLY_HIDX_NORMAL', 'HLY_WCHL_NORMAL', 'years_HLY_WCHL_NORMAL',
       'HLY_WIND_AVGSPD', 'years_HLY_WIND_AVGSPD', 'HLY_WIND_PCTCLM',
       'years_HLY_WIND_PCTCLM', 'HLY_WIND_VCTDIR', 'years_HLY_WIND_VCTDIR',
       'HLY_WIND_VCTSPD', 'years_HLY_WIND_VCTSPD', 'HLY_WIND_1STDIR',
       'years_HLY-WIND-1STDIR', 'HLY_WIND_1STPCT', 'years_HLY_WIND_1STPCT',
       'HLY_WIND_2NDDIR', 'years_HLY_WIND_2NDDIR', 'HLY_WIND_2NDPCT',
       'years_HLY_WIND_2NDPCT']
#DROPPING COLUMNS FOR TESTING
df = df.drop(columns=[ 
        'years_HLY_TEMP_NORMAL',
       'years_HLY_TEMP_10PCTL',  'years_HLY_TEMP_90PCTL',
        'years_HLY_DEWP_NORMAL', 
       'years_HLY_DEWPv10PCTL',  'years_HLY_DEWP_90PCTL',
        'years_HLY_PRES_NORMAL', 
       'years_HLY_PRES_10PCTL', 'years_HLY_PRES_90PCTL',
        'years_HLY_CLDH_NORMAL', 
       'years_HLY_HTDH_NORMAL', 'years_HLY_CLOD_PCTCLR',
        'years_HLY_CLOD_PCTFEW', 
       'years_HLY_CLOD_PCTSCT',  'years_HLY_CLOD_PCTBKN',
        'years_HLY_CLOD_PCTOVC', 
       'years_HLY_HIDX_NORMAL',  'years_HLY_WCHL_NORMAL',
        'years_HLY_WIND_AVGSPD', 
       'years_HLY_WIND_PCTCLM',  'years_HLY_WIND_VCTDIR',
        'years_HLY_WIND_VCTSPD', 
       'years_HLY-WIND-1STDIR', 'years_HLY_WIND_1STPCT',
        'years_HLY_WIND_2NDDIR',
       'years_HLY_WIND_2NDPCT'])
# print("-------------------------------NEW COLUMNS-----------------------------")
print(df)
# print('---------------------------------DESCRIBE------------------------------')
# #describe
# print(df.describe())

                date  HLY_TEMP_NORMAL  HLY_TEMP_10PCTL  HLY_TEMP_90PCTL  \
0     01-01T00:00:00             48.8             44.1             54.0   
1     01-01T01:00:00             48.5             43.0             54.0   
2     01-01T02:00:00             47.6             42.1             53.1   
3     01-01T03:00:00             47.2             41.0             54.0   
4     01-01T04:00:00             46.8             39.9             53.1   
...              ...              ...              ...              ...   
8755  12-31T19:00:00             51.5             46.9             55.9   
8756  12-31T20:00:00             51.0             46.0             55.0   
8757  12-31T21:00:00             50.5             46.0             55.0   
8758  12-31T22:00:00             49.8             45.0             54.0   
8759  12-31T23:00:00             49.4             45.0             54.0   

      HLY_DEWP_NORMAL  HLY_DEWP_10PCTL  HLY_DEWP_90PCTL  HLY_PRES_NORMAL  \
0                43.2  

# Checking

I had some issues with my code because I had deleted problematic rows before. Now I am adding new values in.

In [192]:
df['date']
hours = ['00:00','01:00','02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00',
            '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00']
df.date[0][6:11]
hour = 0
print(df['date'][103:106])
# for i in range(len(df['date'])):
#     if hour >= 24:
#         hour = 0 
#     print(i)
#     if df.date[i][6:11] != hours[hour]:
#         print("Mismatch. i is: ", i)
#         print(df['date'][i][6:11])
#         print("hours should be: ", hours[hour])
#     hour+=1

print("file is fine")

103    01-05T07:00:00
104    01-05T08:00:00
105    01-05T09:00:00
Name: date, dtype: object
file is fine


# The generalized version

In [195]:
def processNOAAFiles(df, location, fileName, working_directory, destination):
    #flag stuff
    measurementFlags = []
    completenessFlags = []
    for column in df.columns:
        if 'meas' in column:
            measurementFlags.append(column)
        elif 'comp' in column:
            completenessFlags.append(column)
    df = checkMeasurement(df)
    df = checkCompleteness(df)
    #drop columns we don't need
    df.drop('STATION', axis=1, inplace=True)
    df.drop('NAME', axis=1, inplace=True)
    df.drop('LATITUDE', axis=1, inplace=True)
    df.drop('LONGITUDE', axis=1, inplace=True)
    df.drop('ELEVATION', axis=1, inplace=True)
    df.drop('month', axis=1, inplace=True)
    df.drop('hour', axis=1, inplace=True)
    df.drop('day', axis=1, inplace=True)

    #checks the flags and removes any rows corresponding with 'M' which means the data is missing
#     measurementFlags = list(df.filter(regex='meas'))
#     for flag in measurementFlags:
#         df = df[~df[flag].str.contains('M')]

#     completenessFlags = list(df.filter(regex='meas'))
#     for flag in completenessFlags:
#         df = df[~df[flag].str.contains('-9999')]

    #get rid of all measurement and completeness flags
    df = df[df.columns.drop(list(df.filter(regex='meas')))]
    df = df[df.columns.drop(list(df.filter(regex='comp')))]

    #rename columns
    df.columns = ['date', 'HLY_TEMP_NORMAL', 'years_HLY_TEMP_NORMAL', 'HLY_TEMP_10PCTL',
       'years_HLY_TEMP_10PCTL', 'HLY_TEMP_90PCTL', 'years_HLY_TEMP_90PCTL',
       'HLY_DEWP_NORMAL', 'years_HLY_DEWP_NORMAL', 'HLY_DEWP_10PCTL',
       'years_HLY_DEWPv10PCTL', 'HLY_DEWP_90PCTL', 'years_HLY_DEWP_90PCTL',
       'HLY_PRES_NORMAL', 'years_HLY_PRES_NORMAL', 'HLY_PRES_10PCTL',
       'years_HLY_PRES_10PCTL', 'HLY_PRES_90PCTL', 'years_HLY_PRES_90PCTL',
       'HLY_CLDH_NORMAL', 'years_HLY_CLDH_NORMAL', 'HLY_HTDH_NORMAL',
       'years_HLY_HTDH_NORMAL', 'HLY_CLOD_PCTCLR', 'years_HLY_CLOD_PCTCLR',
       'HLY_CLOD_PCTFEW', 'years_HLY_CLOD_PCTFEW', 'HLY_CLOD_PCTSCT',
       'years_HLY_CLOD_PCTSCT', 'HLY_CLOD_PCTBKN', 'years_HLY_CLOD_PCTBKN',
       'HLY_CLOD_PCTOVC', 'years_HLY_CLOD_PCTOVC', 'HLY_HIDX_NORMAL',
       'years_HLY_HIDX_NORMAL', 'HLY_WCHL_NORMAL', 'years_HLY_WCHL_NORMAL',
       'HLY_WIND_AVGSPD', 'years_HLY_WIND_AVGSPD', 'HLY_WIND_PCTCLM',
       'years_HLY_WIND_PCTCLM', 'HLY_WIND_VCTDIR', 'years_HLY_WIND_VCTDIR',
       'HLY_WIND_VCTSPD', 'years_HLY_WIND_VCTSPD', 'HLY_WIND_1STDIR',
       'years_HLY-WIND-1STDIR', 'HLY_WIND_1STPCT', 'years_HLY_WIND_1STPCT',
       'HLY_WIND_2NDDIR', 'years_HLY_WIND_2NDDIR', 'HLY_WIND_2NDPCT',
       'years_HLY_WIND_2NDPCT']
    #Dropping these because they don't seem useful
    df = df.drop(columns=[
            'years_HLY_TEMP_NORMAL',
           'years_HLY_TEMP_10PCTL',  'years_HLY_TEMP_90PCTL',
            'years_HLY_DEWP_NORMAL', 
           'years_HLY_DEWPv10PCTL',  'years_HLY_DEWP_90PCTL',
            'years_HLY_PRES_NORMAL', 
           'years_HLY_PRES_10PCTL', 'years_HLY_PRES_90PCTL',
            'years_HLY_CLDH_NORMAL', 
           'years_HLY_HTDH_NORMAL', 'years_HLY_CLOD_PCTCLR',
            'years_HLY_CLOD_PCTFEW', 
           'years_HLY_CLOD_PCTSCT',  'years_HLY_CLOD_PCTBKN',
            'years_HLY_CLOD_PCTOVC', 
           'years_HLY_HIDX_NORMAL',  'years_HLY_WCHL_NORMAL',
            'years_HLY_WIND_AVGSPD', 
           'years_HLY_WIND_PCTCLM',  'years_HLY_WIND_VCTDIR',
            'years_HLY_WIND_VCTSPD', 
           'years_HLY-WIND-1STDIR', 'years_HLY_WIND_1STPCT',
            'years_HLY_WIND_2NDDIR',
           'years_HLY_WIND_2NDPCT'])
    #Rename columns by their location as well
    df.columns = ['date', 'HLY_TEMP_NORMAL' + location, 'HLY_TEMP_10PCTL' + location,
                'HLY_TEMP_90PCTL' + location, 
           'HLY_DEWP_NORMAL' + location, 'HLY_DEWP_10PCTL' + location,
            'HLY_DEWP_90PCTL'  + location,
           'HLY_PRES_NORMAL'  + location, 'HLY_PRES_10PCTL' + location,
            'HLY_PRES_90PCTL' + location, 
           'HLY_CLDH_NORMAL' + location, 'HLY_HTDH_NORMAL' + location,
           'HLY_CLOD_PCTCLR' + location, 
           'HLY_CLOD_PCTFEW' + location, 'HLY_CLOD_PCTSCT' + location,
           'HLY_CLOD_PCTBKN' + location, 
           'HLY_CLOD_PCTOVC' + location, 'HLY_HIDX_NORMAL' + location,
           'HLY_WCHL_NORMAL' + location, 
           'HLY_WIND_AVGSPD' + location,  'HLY_WIND_PCTCLM' + location,
           'HLY_WIND_VCTDIR' + location, 
           'HLY_WIND_VCTSPD' + location, 'HLY_WIND_1STDIR' + location,
            'HLY_WIND_1STPCT' + location,
           'HLY_WIND_2NDDIR' + location,  'HLY_WIND_2NDPCT' + location]
    newCsv = destination + fileName
    df.to_csv(newCsv, encoding='utf-8', index = False)

In [196]:
working_directory = '/Users/yukahatori/A_Fairness/FairnessML_git/NOAA_Data/'
os.chdir(working_directory)
allDataFiles = [f for f in listdir(working_directory) if isfile(join(working_directory, f))] #returns the file names in that directory
files = []
for fileName in allDataFiles:
    print(fileName)
    files.append(fileName)
    df = pd.read_csv(working_directory + fileName)

#00023188 = SD
#00023174 = LA
#00023232 = SAC
#00023152 = burbank
#00023234 = SF
df = pd.read_csv(working_directory + files[0])
srcDir = '/Users/yukahatori/A_Fairness/FairnessML_git/NOAA_Data/'
destDir = '/Users/yukahatori/A_Fairness/FairnessML_git/NOAA_Data_Clean/'
processNOAAFiles(df, '_SD', 'SD.csv',srcDir, destDir)
df = pd.read_csv(working_directory + files[1])
processNOAAFiles(df, '_LA', 'LA.csv',srcDir, destDir)
df = pd.read_csv(working_directory + files[2])
processNOAAFiles(df, '_SAC', 'SAC.csv',srcDir, destDir)
df = pd.read_csv(working_directory + files[4])
processNOAAFiles(df, '_SF', 'SF.csv',srcDir, destDir)

USW00023188.csv
USW00023174.csv
USW00023232.csv
USW00023152.csv
USW00023234.csv
2877    23.0
2878    20.4
2879    21.9
Name: HLY-CLOD-PCTCLR, dtype: float64
2901    22.4
2902    19.6
2903    21.1
Name: HLY-CLOD-PCTCLR, dtype: float64
2904    20.35
2905    18.80
2906    18.80
Name: HLY-CLOD-PCTCLR, dtype: float64
2925    20.9
2926    18.3
2927    20.2
Name: HLY-CLOD-PCTCLR, dtype: float64
2928    19.25
2929    17.50
2930    17.80
Name: HLY-CLOD-PCTCLR, dtype: float64
2949    21.2
2950    17.6
2951    19.6
Name: HLY-CLOD-PCTCLR, dtype: float64
2952    18.6
2953    16.3
2954    16.9
Name: HLY-CLOD-PCTCLR, dtype: float64
2955    16.6
2956    12.2
2957     9.7
Name: HLY-CLOD-PCTCLR, dtype: float64
2973    20.7
2974    17.5
2975    19.7
Name: HLY-CLOD-PCTCLR, dtype: float64
2975    19.7
2976    18.6
2977    16.3
Name: HLY-CLOD-PCTCLR, dtype: float64
2978    17.45
2979    16.70
2980    12.00
Name: HLY-CLOD-PCTCLR, dtype: float64
2979    16.70
2980    12.00
2981    14.35
Name: HLY-CLOD-PCTCLR,

Name: HLY-CLOD-PCTCLR, dtype: float64
3411    9.731250
3412    6.200000
3413    7.965625
Name: HLY-CLOD-PCTCLR, dtype: float64
3412    6.200000
3413    7.965625
3414    7.082812
Name: HLY-CLOD-PCTCLR, dtype: float64
3413    7.965625
3414    7.082812
3415    7.524219
Name: HLY-CLOD-PCTCLR, dtype: float64
3414    7.082812
3415    7.524219
3416    7.303516
Name: HLY-CLOD-PCTCLR, dtype: float64
3428    12.1
3429    12.0
3430     8.7
Name: HLY-CLOD-PCTCLR, dtype: float64
3429    12.00
3430     8.70
3431    10.35
Name: HLY-CLOD-PCTCLR, dtype: float64
3430     8.700
3431    10.350
3432     9.525
Name: HLY-CLOD-PCTCLR, dtype: float64
3431    10.3500
3432     9.5250
3433     9.9375
Name: HLY-CLOD-PCTCLR, dtype: float64
3432    9.52500
3433    9.93750
3434    9.73125
Name: HLY-CLOD-PCTCLR, dtype: float64
3434    9.731250
3435    9.834375
3436    6.200000
Name: HLY-CLOD-PCTCLR, dtype: float64
3435    9.834375
3436    6.200000
3437    8.017188
Name: HLY-CLOD-PCTCLR, dtype: float64
3436    6.200000

Name: HLY-CLOD-PCTCLR, dtype: float64
3797    6.3500
3798    5.3250
3799    5.8375
Name: HLY-CLOD-PCTCLR, dtype: float64
3812    8.4
3813    8.1
3814    8.6
Name: HLY-CLOD-PCTCLR, dtype: float64
3813    8.10
3814    8.60
3815    8.35
Name: HLY-CLOD-PCTCLR, dtype: float64
3814    8.600
3815    8.350
3816    8.475
Name: HLY-CLOD-PCTCLR, dtype: float64
3815    8.3500
3816    8.4750
3817    8.4125
Name: HLY-CLOD-PCTCLR, dtype: float64
3816    8.47500
3817    8.41250
3818    8.44375
Name: HLY-CLOD-PCTCLR, dtype: float64
3818    8.443750
3819    8.428125
3820    4.100000
Name: HLY-CLOD-PCTCLR, dtype: float64
3819    8.428125
3820    4.100000
3821    6.264062
Name: HLY-CLOD-PCTCLR, dtype: float64
3820    4.100000
3821    6.264062
3822    5.182031
Name: HLY-CLOD-PCTCLR, dtype: float64
3821    6.264062
3822    5.182031
3823    5.723047
Name: HLY-CLOD-PCTCLR, dtype: float64
3836    7.8
3837    7.9
3838    8.6
Name: HLY-CLOD-PCTCLR, dtype: float64
3837    7.90
3838    8.60
3839    8.25
Name: HLY-

Name: HLY-CLOD-PCTCLR, dtype: float64
4226    12.743750
4227    12.928125
4228     5.200000
Name: HLY-CLOD-PCTCLR, dtype: float64
4227    12.928125
4228     5.200000
4229     9.064063
Name: HLY-CLOD-PCTCLR, dtype: float64
4228    5.200000
4229    9.064063
4230    7.132031
Name: HLY-CLOD-PCTCLR, dtype: float64
4229    9.064063
4230    7.132031
4231    8.098047
Name: HLY-CLOD-PCTCLR, dtype: float64
4244    13.3
4245    16.5
4246    10.7
Name: HLY-CLOD-PCTCLR, dtype: float64
4245    16.5
4246    10.7
4247    13.6
Name: HLY-CLOD-PCTCLR, dtype: float64
4246    10.70
4247    13.60
4248    12.15
Name: HLY-CLOD-PCTCLR, dtype: float64
4247    13.600
4248    12.150
4249    12.875
Name: HLY-CLOD-PCTCLR, dtype: float64
4248    12.1500
4249    12.8750
4250    12.5125
Name: HLY-CLOD-PCTCLR, dtype: float64
4250    12.51250
4251    12.69375
4252     4.70000
Name: HLY-CLOD-PCTCLR, dtype: float64
4251    12.693750
4252     4.700000
4253     8.696875
Name: HLY-CLOD-PCTCLR, dtype: float64
4252    4.700000

Name: HLY-CLOD-PCTCLR, dtype: float64
4680    11.1500
4681    10.9750
4682    11.0625
Name: HLY-CLOD-PCTCLR, dtype: float64
4682    11.06250
4683    11.01875
4684     5.40000
Name: HLY-CLOD-PCTCLR, dtype: float64
4683    11.018750
4684     5.400000
4685     8.209375
Name: HLY-CLOD-PCTCLR, dtype: float64
4684    5.400000
4685    8.209375
4686    6.804688
Name: HLY-CLOD-PCTCLR, dtype: float64
4685    8.209375
4686    6.804688
4687    7.507031
Name: HLY-CLOD-PCTCLR, dtype: float64
4701    12.8
4702    10.8
4703    10.5
Name: HLY-CLOD-PCTCLR, dtype: float64
4702    10.80
4703    10.50
4704    10.65
Name: HLY-CLOD-PCTCLR, dtype: float64
4703    10.500
4704    10.650
4705    10.575
Name: HLY-CLOD-PCTCLR, dtype: float64
4704    10.6500
4705    10.5750
4706    10.6125
Name: HLY-CLOD-PCTCLR, dtype: float64
4706    10.61250
4707    10.59375
4708     4.90000
Name: HLY-CLOD-PCTCLR, dtype: float64
4707    10.593750
4708     4.900000
4709     7.746875
Name: HLY-CLOD-PCTCLR, dtype: float64
4708    4.

...

7779    6.6
7780    5.9
7781    5.7
Name: HLY-CLOD-PCTBKN, dtype: float64
7803    6.6
7804    5.9
7805    5.7
Name: HLY-CLOD-PCTBKN, dtype: float64
8063    4.5
8064    4.3
8065    6.2
Name: HLY-CLOD-PCTBKN, dtype: float64
8066    5.0
8067    5.8
8068    5.0
Name: HLY-CLOD-PCTBKN, dtype: float64
8068    5.000000
8069    5.266667
8070    4.900000
Name: HLY-CLOD-PCTBKN, dtype: float64
8069    5.266667
8070    4.900000
8071    5.055556
Name: HLY-CLOD-PCTBKN, dtype: float64
8087    5.0
8088    4.9
8089    6.3
Name: HLY-CLOD-PCTBKN, dtype: float64
8088    4.9
8089    6.3
8090    5.4
Name: HLY-CLOD-PCTBKN, dtype: float64
8089    6.300000
8090    5.400000
8091    5.533333
Name: HLY-CLOD-PCTBKN, dtype: float64
8090    5.400000
8091    5.533333
8092    5.744444
Name: HLY-CLOD-PCTBKN, dtype: float64
8092    5.744444
8093    5.559259
8094    5.300000
Name: HLY-CLOD-PCTBKN, dtype: float64
8093    5.559259
8094    5.300000
8095    5.534568
Name: HLY-CLOD-PCTBKN, dtype: float64
8111    4.8
8112    4.

Name: HLY-CLOD-PCTOVC, dtype: float64
99     29.520576
100    29.508916
101    29.520530
Name: HLY-CLOD-PCTOVC, dtype: float64
100    29.508916
101    29.520530
102    29.516674
Name: HLY-CLOD-PCTOVC, dtype: float64
101    29.520530
102    29.516674
103    29.515374
Name: HLY-CLOD-PCTOVC, dtype: float64
102    29.516674
103    29.515374
104    29.517526
Name: HLY-CLOD-PCTOVC, dtype: float64
103    29.515374
104    29.517526
105    29.516525
Name: HLY-CLOD-PCTOVC, dtype: float64
116    29.5
117    30.8
118    30.0
Name: HLY-CLOD-PCTOVC, dtype: float64
117    30.8
118    30.0
119    30.1
Name: HLY-CLOD-PCTOVC, dtype: float64
118    30.0
119    30.1
120    30.3
Name: HLY-CLOD-PCTOVC, dtype: float64
119    30.100000
120    30.300000
121    30.133333
Name: HLY-CLOD-PCTOVC, dtype: float64
120    30.300000
121    30.133333
122    30.177778
Name: HLY-CLOD-PCTOVC, dtype: float64
121    30.133333
122    30.177778
123    30.203704
Name: HLY-CLOD-PCTOVC, dtype: float64
122    30.177778
123    30.2

Name: HLY-CLOD-PCTOVC, dtype: float64
430    31.500000
431    30.333333
432    30.744444
Name: HLY-CLOD-PCTOVC, dtype: float64
431    30.333333
432    30.744444
433    30.859259
Name: HLY-CLOD-PCTOVC, dtype: float64
432    30.744444
433    30.859259
434    30.645679
Name: HLY-CLOD-PCTOVC, dtype: float64
433    30.859259
434    30.645679
435    30.749794
Name: HLY-CLOD-PCTOVC, dtype: float64
434    30.645679
435    30.749794
436    30.751578
Name: HLY-CLOD-PCTOVC, dtype: float64
435    30.749794
436    30.751578
437    30.715684
Name: HLY-CLOD-PCTOVC, dtype: float64
436    30.751578
437    30.715684
438    30.739018
Name: HLY-CLOD-PCTOVC, dtype: float64
437    30.715684
438    30.739018
439    30.735427
Name: HLY-CLOD-PCTOVC, dtype: float64
438    30.739018
439    30.735427
440    30.730043
Name: HLY-CLOD-PCTOVC, dtype: float64
439    30.735427
440    30.730043
441    30.734829
Name: HLY-CLOD-PCTOVC, dtype: float64
440    30.730043
441    30.734829
442    30.733433
Name: HLY-CLOD-PCTOVC

Name: HLY-CLOD-PCTOVC, dtype: float64
7803    19.7
7804    20.1
7805    21.5
Name: HLY-CLOD-PCTOVC, dtype: float64
8063    23.0
8064    25.1
8065    25.7
Name: HLY-CLOD-PCTOVC, dtype: float64
8066    24.6
8067    26.3
8068    27.8
Name: HLY-CLOD-PCTOVC, dtype: float64
8068    27.800000
8069    26.233333
8070    29.600000
Name: HLY-CLOD-PCTOVC, dtype: float64
8069    26.233333
8070    29.600000
8071    27.877778
Name: HLY-CLOD-PCTOVC, dtype: float64
8087    23.5
8088    25.2
8089    26.3
Name: HLY-CLOD-PCTOVC, dtype: float64
8088    25.2
8089    26.3
8090    25.0
Name: HLY-CLOD-PCTOVC, dtype: float64
8089    26.3
8090    25.0
8091    25.5
Name: HLY-CLOD-PCTOVC, dtype: float64
8090    25.0
8091    25.5
8092    25.6
Name: HLY-CLOD-PCTOVC, dtype: float64
8092    25.600000
8093    25.366667
8094    30.800000
Name: HLY-CLOD-PCTOVC, dtype: float64
8093    25.366667
8094    30.800000
8095    27.255556
Name: HLY-CLOD-PCTOVC, dtype: float64
8111    23.5
8112    25.7
8113    26.6
Name: HLY-CLOD-P

102    21.9
103    11.6
104    10.1
Name: HLY-CLOD-PCTCLR, dtype: float64
126    21.5
127    11.3
128     9.6
Name: HLY-CLOD-PCTCLR, dtype: float64
150    21.4
151    11.4
152    10.2
Name: HLY-CLOD-PCTCLR, dtype: float64
198    22.5
199    12.9
200    12.2
Name: HLY-CLOD-PCTCLR, dtype: float64
221    25.6
222    22.9
223    13.3
Name: HLY-CLOD-PCTCLR, dtype: float64
222    22.9
223    13.3
224    13.3
Name: HLY-CLOD-PCTCLR, dtype: float64
245    26.2
246    23.6
247    14.1
Name: HLY-CLOD-PCTCLR, dtype: float64
246    23.6
247    14.1
248    14.1
Name: HLY-CLOD-PCTCLR, dtype: float64
270    25.0
271    15.0
272    14.7
Name: HLY-CLOD-PCTCLR, dtype: float64
292    24.8
293    26.8
294    24.2
Name: HLY-CLOD-PCTCLR, dtype: float64
294    24.2
295    24.2
296    13.9
Name: HLY-CLOD-PCTCLR, dtype: float64
318    24.1
319    13.9
320    13.3
Name: HLY-CLOD-PCTCLR, dtype: float64
5187    14.8
5188    13.1
5189     9.6
Name: HLY-CLOD-PCTCLR, dtype: float64
5211    15.3
5212    13.1
5213     

In [ ]:
def processNOAAFiles_justOneYear(df, location, fileName, working_directory, destination):
    #flag stuff
    measurementFlags = []
    completenessFlags = []
    for column in df.columns:
        if 'meas' in column:
            measurementFlags.append(column)
        elif 'comp' in column:
            completenessFlags.append(column)
    df = checkMeasurement(df)
    df = checkCompleteness(df)
    #drop columns we don't need
    df.drop('STATION', axis=1, inplace=True)
    df.drop('NAME', axis=1, inplace=True)
    df.drop('LATITUDE', axis=1, inplace=True)
    df.drop('LONGITUDE', axis=1, inplace=True)
    df.drop('ELEVATION', axis=1, inplace=True)
    df.drop('month', axis=1, inplace=True)
    df.drop('hour', axis=1, inplace=True)
    df.drop('day', axis=1, inplace=True)

    #checks the flags and removes any rows corresponding with 'M' which means the data is missing
#     measurementFlags = list(df.filter(regex='meas'))
#     for flag in measurementFlags:
#         df = df[~df[flag].str.contains('M')]

#     completenessFlags = list(df.filter(regex='meas'))
#     for flag in completenessFlags:
#         df = df[~df[flag].str.contains('-9999')]

    #get rid of all measurement and completeness flags
    df = df[df.columns.drop(list(df.filter(regex='meas')))]
    df = df[df.columns.drop(list(df.filter(regex='comp')))]

    #rename columns
    df.columns = ['date', 'HLY_TEMP_NORMAL', 'years_HLY_TEMP_NORMAL', 'HLY_TEMP_10PCTL',
       'years_HLY_TEMP_10PCTL', 'HLY_TEMP_90PCTL', 'years_HLY_TEMP_90PCTL',
       'HLY_DEWP_NORMAL', 'years_HLY_DEWP_NORMAL', 'HLY_DEWP_10PCTL',
       'years_HLY_DEWPv10PCTL', 'HLY_DEWP_90PCTL', 'years_HLY_DEWP_90PCTL',
       'HLY_PRES_NORMAL', 'years_HLY_PRES_NORMAL', 'HLY_PRES_10PCTL',
       'years_HLY_PRES_10PCTL', 'HLY_PRES_90PCTL', 'years_HLY_PRES_90PCTL',
       'HLY_CLDH_NORMAL', 'years_HLY_CLDH_NORMAL', 'HLY_HTDH_NORMAL',
       'years_HLY_HTDH_NORMAL', 'HLY_CLOD_PCTCLR', 'years_HLY_CLOD_PCTCLR',
       'HLY_CLOD_PCTFEW', 'years_HLY_CLOD_PCTFEW', 'HLY_CLOD_PCTSCT',
       'years_HLY_CLOD_PCTSCT', 'HLY_CLOD_PCTBKN', 'years_HLY_CLOD_PCTBKN',
       'HLY_CLOD_PCTOVC', 'years_HLY_CLOD_PCTOVC', 'HLY_HIDX_NORMAL',
       'years_HLY_HIDX_NORMAL', 'HLY_WCHL_NORMAL', 'years_HLY_WCHL_NORMAL',
       'HLY_WIND_AVGSPD', 'years_HLY_WIND_AVGSPD', 'HLY_WIND_PCTCLM',
       'years_HLY_WIND_PCTCLM', 'HLY_WIND_VCTDIR', 'years_HLY_WIND_VCTDIR',
       'HLY_WIND_VCTSPD', 'years_HLY_WIND_VCTSPD', 'HLY_WIND_1STDIR',
       'years_HLY-WIND-1STDIR', 'HLY_WIND_1STPCT', 'years_HLY_WIND_1STPCT',
       'HLY_WIND_2NDDIR', 'years_HLY_WIND_2NDDIR', 'HLY_WIND_2NDPCT',
       'years_HLY_WIND_2NDPCT']
    #Dropping these because they don't seem useful
    df = df.drop(columns=[
            'years_HLY_TEMP_NORMAL',
           'years_HLY_TEMP_10PCTL',  'years_HLY_TEMP_90PCTL',
            'years_HLY_DEWP_NORMAL', 
           'years_HLY_DEWPv10PCTL',  'years_HLY_DEWP_90PCTL',
            'years_HLY_PRES_NORMAL', 
           'years_HLY_PRES_10PCTL', 'years_HLY_PRES_90PCTL',
            'years_HLY_CLDH_NORMAL', 
           'years_HLY_HTDH_NORMAL', 'years_HLY_CLOD_PCTCLR',
            'years_HLY_CLOD_PCTFEW', 
           'years_HLY_CLOD_PCTSCT',  'years_HLY_CLOD_PCTBKN',
            'years_HLY_CLOD_PCTOVC', 
           'years_HLY_HIDX_NORMAL',  'years_HLY_WCHL_NORMAL',
            'years_HLY_WIND_AVGSPD', 
           'years_HLY_WIND_PCTCLM',  'years_HLY_WIND_VCTDIR',
            'years_HLY_WIND_VCTSPD', 
           'years_HLY-WIND-1STDIR', 'years_HLY_WIND_1STPCT',
            'years_HLY_WIND_2NDDIR',
           'years_HLY_WIND_2NDPCT'])
    #Rename columns by their location as well
    df.columns = ['date', 'HLY_TEMP_NORMAL' + location, 'HLY_TEMP_10PCTL' + location,
                'HLY_TEMP_90PCTL' + location, 
           'HLY_DEWP_NORMAL' + location, 'HLY_DEWP_10PCTL' + location,
            'HLY_DEWP_90PCTL'  + location,
           'HLY_PRES_NORMAL'  + location, 'HLY_PRES_10PCTL' + location,
            'HLY_PRES_90PCTL' + location, 
           'HLY_CLDH_NORMAL' + location, 'HLY_HTDH_NORMAL' + location,
           'HLY_CLOD_PCTCLR' + location, 
           'HLY_CLOD_PCTFEW' + location, 'HLY_CLOD_PCTSCT' + location,
           'HLY_CLOD_PCTBKN' + location, 
           'HLY_CLOD_PCTOVC' + location, 'HLY_HIDX_NORMAL' + location,
           'HLY_WCHL_NORMAL' + location, 
           'HLY_WIND_AVGSPD' + location,  'HLY_WIND_PCTCLM' + location,
           'HLY_WIND_VCTDIR' + location, 
           'HLY_WIND_VCTSPD' + location, 'HLY_WIND_1STDIR' + location,
            'HLY_WIND_1STPCT' + location,
           'HLY_WIND_2NDDIR' + location,  'HLY_WIND_2NDPCT' + location]
    newCsv = destination + fileName
    df.to_csv(newCsv, encoding='utf-8', index = False)